# Análisis e Implementación en SparkQL

* Entrega hasta el 24 de mayo en https://forms.gle/aRiWtjayausRFCTd8

* Los dos conjuntos de datos entregados, CSV separados por coma, Evaluación del Pitch (2021.04.13 ISoftware).csv y Asistencia del Pitch (2021.04.13 ISoftware).csv proceden de dos encuestas realizadas con Google Forms.

* Los archivos proceden de las evaluaciones de los pitch (exposiciones cortas de negocio) que realizan los estudiantes de la asignatura de Ingeniería de Software. Los estudiantes están organizados en equipos. Cada equipo realiza una presentación de máximo 15 minutos, una vez inicia la presentación todos los estudiantes deben registrar su asistencia lo cual queda registrado en 'Asistencia del Pitch (2021.04.13 ISoftware).csv' incluidos los miembros del equipo.

* El archivo `Asistencia del Pitch (2021.04.13 ISoftware).csv` contiene las columnas: `"Marca temporal"` es tiempo dado en fecha y hora, `"Nombre de usuario"` es el correo electrónico del estudiante y es texto,`"Equipo al que perteneces:"` equipo de trabajo al que pertenece el estudiante también es texto y `"Equipo que va a exponer:"` equipo que el estudiante va a tender a su presentación.

* El archivo `Evaluación del Pitch (2021.04.13 ISoftware).csv` contiene las columnas `"Marca temporal"` es tiempo dado en fecha y hora, `"Nombre de usuario"` es el correo electrónico del estudiante y es texto, `"Equipo que vas a evaluar:"` equipo que ha expuesto y que va a ser evaluado por cada estudiante que no sea integrante; a continuación, se tienen las siguientes columnas que corresponden a la evaluación de los respectivos ítems:

    1. "Introducción: El equipo responde adecuadamente ¿Quiénes son y por qué están aquí?",
    2. "Equipo: El equipo responde adecuadamente ¿Quiénes están detrás de la idea y cuál es su función?",
    3. "Problema: El equipo responde adecuadamente ¿Qué problema resolverá?, ¿es realmente un problema?",
    4. "Ventajas: El equipo responde adecuadamente ¿Por qué su solución es especial?, ¿qué la hace distinta de otras?",
    5. "Solución: El equipo responde adecuadamente ¿Cómo piensa resolver el problema?",
    6. "Producto: El equipo responde adecuadamente ¿Cómo funciona el producto o servicio? Muestra algunos ejemplos.",
    7. "Tracción: El equipo responde adecuadamente si cuenta con clientes que demuestran potencial.",
    8. "Mercado: El equipo responde conoce, o por lo menos intentar predecir, el tamaño del mercado que impactará.",
    9. "Competencia: El equipo responde adecuadamente ¿Cuáles son las soluciones alternativas al problema que plantea?",
    10. "Modelo de negocio: El equipo responde adecuadamente ¿Cómo hará dinero? ",
    11. "Inversión: El equipo responde adecuadamente ¿Cuál es su presupuesto y cuánto espera ganar?",
    12. "Contacto: El equipo deja los datos al cliente y muestra cómo pueden contactarle.",
    13. "Exposición: ¿Qué tan coordinados estaban los expositores?",
    14. "Exposición: ¿Los expositores se expresaron con claridad y se hicieron entender?",
    15. "Exposición: Las diapositivas son claras y coherentes y apoyaron adecuadamente la exposición.",
    * "Suponiendo que eres inversionista, ¿Estarías dispuesto a invertir dinero en este equipo? (esta pregunta no se pondera en la nota)",
    * "Observaciones para el equipo, estas observaciones las debe considerar el equipo para mejorar la siguiente presentación."


* Cada ítem se evalúa con la siguiente escala: 0. Ausente; 1. Deficiente; 2. Regular; 3. Aceptable; 4. Bueno; 5. Excelente

* Carge los datos, cada archivo en una tabla SparkSQL y responda cada una de las consultas dadas en cada celda. Tenga en cuenta que algunas consultas pueden tener como resultado el vacío.

# Integrantes del equipo
1. Ana Maria Gaona Gomez
2. Julian Palacio Roldan
3. Natalia Carmona Restrepo
4. Tomas Vergara Cardona
5.

### Importante: Por favor ejecutar este notebook en la versión Pseudo del contenedor de Spark

In [1]:
# Agrege acá el código para importar las librerias

# La libreria para "encontrar el sevicio" de Spark
import findspark
findspark.init()

# Librerias para "gestionar el servicio" de Spark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, SparkSession, DataFrame
import pyspark.sql.functions as F
from functools import reduce


# Creamos una aplicación Spark en el Servicio
# Tenga cuidado con las tildes o caracteres especiales en el nombre de la app
AppSpark = SparkConf().setAppName("Evaluacion iSofware")

# definimos un espacio o contexto para la App
ContextoSpark=SparkContext(conf=AppSpark)

# inicio una sesión en el espacio de la App
SesionSpark = SparkSession(ContextoSpark)

# inicio del espacio o contexto SQL
ContextoSql = SQLContext(sparkContext=ContextoSpark, sparkSession=SesionSpark)

In [2]:
# 1. Cargue los datos en la carpeta datalake y luego del /datalake al HDFS (Hadoop File System)
# Recuerda usar ! para ejecutar el comando en el shell. 
# Tu código a continuación...

!hadoop fs -rm -r "Asistencia.csv"
!hadoop fs -rm -r "Evaluacion.csv"
!hadoop fs -put "/datalake/Asistencia.csv"
!hadoop fs -put "/datalake/Evaluacion.csv"
!hadoop fs -ls

rm: `Asistencia.csv': No such file or directory
rm: `Evaluacion.csv': No such file or directory
Found 2 items
-rw-r--r--   1 root supergroup      93121 2021-05-24 04:17 Asistencia.csv
-rw-r--r--   1 root supergroup     111944 2021-05-24 04:17 Evaluacion.csv


In [3]:
# 2. Cree dos tablas SparkSQL y almacene el csv en su correspondiente tabla.
# Observación: tenga especial cuidado con los encabezados de los archivos CSV.
# Usted puede considerar cambiar los encabezados de los CSV originales
# Tu código a continuación...

ContextoSql.sql("DROP TABLE IF EXISTS asistencia")
ContextoSql.sql("DROP TABLE IF EXISTS evaluacion")
ContextoSql.sql("DROP TABLE IF EXISTS evaluacion2")
ContextoSql.sql("""
CREATE TABLE IF NOT EXISTS 
    asistencia (
        fecha TIMESTAMP,
        email STRING,
        equipo STRING,
        expositor STRING
        ) 
USING com.databricks.spark.csv 
OPTIONS (
    path 'Asistencia.csv', 
    header='true',
    timestampFormat=\"yyyy/MM/dd HH:mm:ss\")
    
""");

ContextoSql.sql("""
CREATE TABLE IF NOT EXISTS 
    evaluacion (
        fecha TIMESTAMP,
        email STRING,
        evaluado STRING,
        introduccion INT,
        equipo INT,
        problema INT,
        ventajas INT,
        solucion INT,
        producto INT,
        traccion INT,
        mercado INT,
        competencia INT,
        modelo INT,
        inversion INT,
        contacto INT,
        exposicion1 INT,
        exposicion2 INT,
        exposicion3 INT,
        invertiria STRING,
        observaciones STRING
        ) 
USING com.databricks.spark.csv 
OPTIONS (
    path 'Evaluacion.csv', 
    header='true',
    timestampFormat=\"yyyy/MM/dd HH:mm:ss\")
""");


In [12]:
ContextoSql.sql("SHOW TABLES").show()

#Se crean los dataframe
asistencia = ContextoSql.sql("SELECT * FROM asistencia")
evaluacion = ContextoSql.sql("SELECT * FROM evaluacion").na.drop("all")

+--------+----------+-----------+
|database| tableName|isTemporary|
+--------+----------+-----------+
| default|asistencia|      false|
| default|evaluacion|      false|
+--------+----------+-----------+



In [13]:
# 2. Consulte el listado total de estudiantes (correos electrónicos) del 
# curso de Ingeniería de Software, ordenados alfabéticamente
# Tu código a continuación...

asistencia.select('email').distinct().orderBy('email').show(100, truncate=False)
#ContextoSql.sql("SELECT DISTINCT email from asistencia ORDER BY email").show(100)

+-------------------------+
|email                    |
+-------------------------+
|abanolc@unal.edu.co      |
|acastrillonv@unal.edu.co |
|aflemag@unal.edu.co      |
|anapariciom@unal.edu.co  |
|angutierrezb@unal.edu.co |
|anoriega@unal.edu.co     |
|auarbelaeza@unal.edu.co  |
|bocampo@unal.edu.co      |
|cgiraldo@unal.edu.co     |
|cjfunezg@unal.edu.co     |
|cquinchiar@unal.edu.co   |
|dadazam@unal.edu.co      |
|daestradam@unal.edu.co   |
|davgarciava@unal.edu.co  |
|dballesteroso@unal.edu.co|
|dbrito@unal.edu.co       |
|dcadavid@unal.edu.co     |
|dcardonaal@unal.edu.co   |
|dchavarriar@unal.edu.co  |
|dgarciabl@unal.edu.co    |
|dgerenal@unal.edu.co     |
|dgiraldolo@unal.edu.co   |
|dguardia@unal.edu.co     |
|dosoriom@unal.edu.co     |
|eapenad@unal.edu.co      |
|elondonoc@unal.edu.co    |
|emflorezb@unal.edu.co    |
|emlopezr@unal.edu.co     |
|eporrasm@unal.edu.co     |
|esgarciac@unal.edu.co    |
|favasquezj@unal.edu.co   |
|fguerrerot@unal.edu.co   |
|fmiranda@unal.edu.c

In [14]:
# 3. Consulte la cantidad de asistencias registradas por estudiante; además, la fecha y hora de la primera asistencia
# y la fecha y hora de la última asistencia
# Tu código a continuación..
cantidad_asistencia2=asistencia.groupBy('email').agg(F.count("email").alias('número de asistencias'),F.max("fecha").alias('última asistencia'),F.min("fecha").alias('Primera asistencia')).show(truncate=False)

+-------------------------+---------------------+-------------------+-------------------+
|email                    |número de asistencias|última asistencia  |Primera asistencia |
+-------------------------+---------------------+-------------------+-------------------+
|samolinap@unal.edu.co    |10                   |2021-04-13 09:37:26|2021-04-08 08:11:43|
|jruedat@unal.edu.co      |8                    |2021-04-13 09:41:27|2021-04-08 08:12:55|
|nvalenciat@unal.edu.co   |8                    |2021-04-13 09:39:34|2021-04-08 08:10:18|
|serendona@unal.edu.co    |8                    |2021-04-13 09:40:25|2021-04-08 08:10:47|
|jgutierrezlo@unal.edu.co |8                    |2021-04-13 09:39:06|2021-04-08 08:12:09|
|dgerenal@unal.edu.co     |8                    |2021-04-13 09:41:27|2021-04-08 08:10:03|
|cjfunezg@unal.edu.co     |8                    |2021-04-13 09:41:46|2021-04-08 08:12:15|
|emflorezb@unal.edu.co    |8                    |2021-04-13 09:45:49|2021-04-08 08:14:04|
|dchavarri

In [15]:
# 4. Consulte el listado de estudiantes que asistieron a 2 presentaciones o menos (una).
# Tu código a continuación...
pocas_asistencias=asistencia.groupBy('email').agg(F.count("email").alias('número de asistencias'))
final_asistencia=pocas_asistencias.select(['email','número de asistencias']).filter(pocas_asistencias['número de asistencias']<=2)
final_asistencia.show(10)

+--------------------+---------------------+
|               email|número de asistencias|
+--------------------+---------------------+
|jdroldano@unal.ed...|                    1|
+--------------------+---------------------+



In [16]:
# 5. Consulte el listado de estudiantes que no asistieron a ninguna presentación.
# Tu código a continuación...
pocas_asistencias=asistencia.groupBy('email').agg(F.count("email").alias('número de asistencias'))
final_asistencia=pocas_asistencias.select(['email','número de asistencias']).filter(pocas_asistencias['número de asistencias']<1)
final_asistencia.show(10)

+-----+---------------------+
|email|número de asistencias|
+-----+---------------------+
+-----+---------------------+



In [17]:
# 6. Consulte los integrantes por cada equipo al que pertenecen.
# Tu código a continuación...
integrantes = asistencia.select(F.regexp_replace(F.col('equipo'), 'Pertenezco a: ', '').alias('equipo'),'email').orderBy('email').orderBy('equipo').distinct()
integrantes.show(100, truncate=False)

+-----------------------------------------------------+-------------------------+
|equipo                                               |email                    |
+-----------------------------------------------------+-------------------------+
|Administración de Inmuebles                          |lumarinb@unal.edu.co     |
|Administración de Inmuebles                          |framirezf@unal.edu.co    |
|Administración de Inmuebles                          |samolinap@unal.edu.co    |
|Administración de Inmuebles                          |sednarvaezna@unal.edu.co |
|Administración de Inmuebles                          |nsalinas@unal.edu.co     |
|Administración de Inmuebles                          |eapenad@unal.edu.co      |
|Administración de Inmuebles                          |jortize@unal.edu.co      |
|Administración de Inmuebles                          |emflorezb@unal.edu.co    |
|Administración de Inmuebles                          |rmbuilesm@unal.edu.co    |
|Administración 

In [18]:
# 7. Consulte la cantidad de asistentes por presentación, sin considerar los asistentes que pertenecen 
# al equipo que realizó la presentación.
# Tu código a continuación...
asistencia.select(F.regexp_replace(F.col('expositor'), 'Expone: ', '').alias('expositor'),
                  F.regexp_replace(F.col('equipo'), 'Pertenezco a: ', '').alias('equipo')
                 ).select('expositor','equipo').filter(F.col('expositor')!=F.col('equipo')).orderBy('expositor').groupBy("expositor").agg(F.count("expositor").alias('Asistencias')).show(truncate=False)


+-----------------------------------------------------+-----------+
|expositor                                            |Asistencias|
+-----------------------------------------------------+-----------+
|Administración de Inmuebles                          |70         |
|Deserción de estudiantes                             |3          |
|Gestión de Menús Ejecutivos                          |72         |
|Gestión de Residencias y Habitaciones Universitarias.|72         |
|Gestión de programas curriculares                    |74         |
|Lo tengo y te lo presto.                             |75         |
|Mi Sanandresito                                      |72         |
|Servicios a domicilio                                |76         |
|Te lo cambio.                                        |75         |
+-----------------------------------------------------+-----------+



In [19]:
# 8. Consutar cuáles integrantes evaluaron a su propio equipo. Estas evaluaciones no serán válidas, pues un
# integrante no puede evaluar a su propio equipo.
# Tu código a continuación...
ListadoEquipos=asistencia.select('email',F.regexp_replace(F.col('equipo'), 'Pertenezco a: ', '').alias('equipo')).distinct().orderBy('email')
eval_no_validas = ListadoEquipos.join(evaluacion,evaluacion.email == ListadoEquipos.email,how='full').select(evaluacion.email, 
                                                                                           ListadoEquipos.equipo,
                                                                                           evaluacion.evaluado).filter(ListadoEquipos.equipo==evaluacion.evaluado).orderBy(evaluacion.email)
eval_no_validas.show(truncate=False)

+-----------------------+-----------------------------------------------------+-----------------------------------------------------+
|email                  |equipo                                               |evaluado                                             |
+-----------------------+-----------------------------------------------------+-----------------------------------------------------+
|anapariciom@unal.edu.co|Servicios a domicilio                                |Servicios a domicilio                                |
|daestradam@unal.edu.co |Lo tengo y te lo presto.                             |Lo tengo y te lo presto.                             |
|hriveraa@unal.edu.co   |Gestión de Menús Ejecutivos                          |Gestión de Menús Ejecutivos                          |
|jhcordoba@unal.edu.co  |Administración de Inmuebles                          |Administración de Inmuebles                          |
|jruedat@unal.edu.co    |Servicios a domicilio                

In [20]:
# 9. Consultar la nota promedio por cada ítem (1 al 15), y la nota promedio total del cada equipo. Recuerde que no
# son válidas las evaluaciones realizadas por los miembros del mismo equipo.
# Tu código a continuación...

#Agrgar a cada alumno que haya realizado una evluación el equipo al que pertenece
eval_eq = evaluacion.join(eval_no_validas.withColumnRenamed("equipo", "equipo_propio"), ['email', 'evaluado'], 'left')
#Filtrar las evaluaciones validas, ya que cuando un alumno evalua su propio equipo no cuenta.
eval_valida = eval_eq.filter(eval_eq.equipo_propio.isNull())
# se retiran las 8 evaluaciones inválidas del punto anterior, pasando de 574 datos de evaluaciones a 566
print(evaluacion.count(), eval_valida.count())

574 566


In [21]:
#Nota Promedio por Item
from functools import reduce
from operator import add
print("---------Nota Promedio por Cada integrante---------")
cols_num = ['introduccion', 'equipo', 'problema', 'ventajas', 'solucion', 'producto', 'traccion', 'mercado',
            'competencia', 'modelo', 'inversion', 'contacto', 'exposicion1', 'exposicion2', 'exposicion3',]
print('Columas:', len(cols_num))
n = len(cols_num)
rowMean = (reduce(add, (F.col(x) for x in cols_num)) / n).alias("Nota_Promedio")
eval_valida.select("evaluado", rowMean).show(truncate=False)

print("---------Nota Promedio por Cada item---------")
promedio_item = eval_valida.describe()
promedio_item = promedio_item.filter(promedio_item["summary"] == "mean")
print("--Se muestra en tres partes para facilitar visualización--")
promedio_item.select(["summary"] + cols_num[:5]).show()
promedio_item.select(["summary"] + cols_num[5:10]).show()
promedio_item.select(["summary"] + cols_num[10:]).show()

---------Nota Promedio por Cada integrante---------
Columas: 15
+---------------------------------+------------------+
|evaluado                         |Nota_Promedio     |
+---------------------------------+------------------+
|Servicios a domicilio            |3.8               |
|Mi Sanandresito                  |4.133333333333334 |
|Te lo cambio.                    |4.533333333333333 |
|Servicios a domicilio            |4.4               |
|Administración de Inmuebles      |4.2               |
|Gestión de programas curriculares|4.733333333333333 |
|Administración de Inmuebles      |4.533333333333333 |
|Te lo cambio.                    |4.466666666666667 |
|Servicios a domicilio            |4.733333333333333 |
|Te lo cambio.                    |4.866666666666666 |
|Administración de Inmuebles      |4.533333333333333 |
|Mi Sanandresito                  |4.4               |
|Gestión de Menús Ejecutivos      |4.133333333333334 |
|Te lo cambio.                    |3.7333333333333334|
|

In [22]:
#Nota Promedio Total de cada Equipo

eval_mod = eval_valida.withColumn('Promedio Alumno',sum([eval_valida[i] for i in cols_num])/ n)
eval_mod.groupby("evaluado").avg("Promedio Alumno").alias("Promedio Equipo").show(truncate=False)

+-----------------------------------------------------+--------------------+
|evaluado                                             |avg(Promedio Alumno)|
+-----------------------------------------------------+--------------------+
|Servicios a domicilio                                |4.435238095238095   |
|Lo tengo y te lo presto.                             |4.479047619047619   |
|Gestión de Residencias y Habitaciones Universitarias.|4.475490196078432   |
|Te lo cambio.                                        |4.2854460093896725  |
|Administración de Inmuebles                          |4.172549019607844   |
|Deserción de estudiantes                             |4.766666666666667   |
|Gestión de Menús Ejecutivos                          |4.360185185185184   |
|Mi Sanandresito                                      |4.202739726027398   |
|Gestión de programas curriculares                    |4.391666666666665   |
+-----------------------------------------------------+--------------------+

In [23]:
# 10. Consulte el mejor equipo evaluado por cada ítem (según nota promedio. En caso de empate mostrar todos los empatados) 
# y el mejor equipo según el promedio total.
# Tu código a continuación...

# se agrupa por evaluado
eval_avg = eval_valida.groupBy('evaluado').avg()


print('--------- Mejor equipo por Item ---------')
result_list = []
for c in eval_avg.columns[1:]:
    max_val = eval_avg.agg({c:"max"}).collect()[0]
    max_val = max_val[f"max({c})"]

    result_list.append(eval_avg.select(F.lit(c).alias('item'), 
                             F.col('evaluado').alias('Mejor Equipo'), 
                             F.col(c).alias('max(avg)'))
                    .filter(eval_avg[c] == max_val))
    
    
    
reduce(DataFrame.unionAll, result_list).show()

print('--------- se comprueban algunos resultados ---------')
eval_avg.select(eval_avg.columns[:5]).show()


print('--------- Mejor equipo según el promedio del total ---------')
avg_func = sum(F.col(x) for x in eval_avg.columns[1:])/len(eval_avg.columns[1:])
puntaje_equipos = eval_avg.withColumn('Promedio Total', avg_func).select(F.col('evaluado').alias('equipo'),F.col('Promedio Total'))
puntaje_equipos.orderBy('Promedio Total', ascending=False).show()


--------- Mejor equipo por Item ---------
+-----------------+--------------------+-----------------+
|             item|        Mejor Equipo|         max(avg)|
+-----------------+--------------------+-----------------+
|avg(introduccion)|Gestión de Reside...|4.544117647058823|
|      avg(equipo)|Deserción de estu...|              5.0|
|    avg(problema)|Deserción de estu...|              5.0|
|    avg(ventajas)|Deserción de estu...|              5.0|
|    avg(solucion)|Servicios a domic...|4.514285714285714|
|    avg(producto)|Deserción de estu...|              5.0|
|    avg(traccion)|Deserción de estu...|              5.0|
|     avg(mercado)|Gestión de Reside...|4.676470588235294|
| avg(competencia)|Deserción de estu...|              5.0|
|      avg(modelo)|Gestión de Menús ...|4.638888888888889|
|   avg(inversion)|Gestión de progra...|4.513888888888889|
|    avg(contacto)|Deserción de estu...|              5.0|
| avg(exposicion1)|       Te lo cambio.|4.577464788732394|
| avg(exposici

In [24]:
# 11. Consulte el peor equipo evaluado por cada ítem (según nota promedio. En caso de empate mostrar todos los empatados) 
# y el peor equipo según el promedio total.
# Tu código a continuación...


print('--------- Peor equipo por Item ---------')
result_list = []
for c in eval_avg.columns[1:]:
    min_val = eval_avg.agg({c:"min"}).collect()[0]
    min_val = min_val[f"min({c})"]

    result_list.append(eval_avg.select(F.lit(c).alias('item'), 
                             F.col('evaluado').alias('Peor Equipo'), 
                             F.col(c).alias('min(avg)'))
                    .filter(eval_avg[c] == min_val))
    
reduce(DataFrame.unionAll, result_list).show()

print('--------- se comprueban algunos resultados ---------')
eval_avg.select(eval_avg.columns[:5]).show()


print('--------- Peor equipo según el promedio del total ---------')
puntaje_equipos.orderBy('Promedio Total', ascending=True).show()


--------- Peor equipo por Item ---------
+-----------------+--------------------+------------------+
|             item|         Peor Equipo|          min(avg)|
+-----------------+--------------------+------------------+
|avg(introduccion)|Administración de...| 4.102941176470588|
|      avg(equipo)|     Mi Sanandresito| 3.904109589041096|
|    avg(problema)|Administración de...|3.9558823529411766|
|    avg(ventajas)|Gestión de progra...|3.9027777777777777|
|    avg(solucion)|Administración de...| 4.176470588235294|
|    avg(producto)|Gestión de Menús ...| 4.027777777777778|
|    avg(traccion)|Administración de...| 4.029411764705882|
|     avg(mercado)|       Te lo cambio.| 4.084507042253521|
| avg(competencia)|     Mi Sanandresito|3.5205479452054793|
|      avg(modelo)|Servicios a domic...| 4.285714285714286|
|   avg(inversion)|       Te lo cambio.| 3.619718309859155|
|    avg(contacto)|Administración de...| 4.132352941176471|
| avg(exposicion1)|     Mi Sanandresito| 4.260273972602739|

In [216]:
# 12. Consulte la lista de estudiantes con la correspondiente nota obtenida en la presentación 
# (nota promedio total de la evaluación obtenida por el equipo)
# Tu código a continuación...

puntaje_equipos.show(10)
integrantes.join(puntaje_equipos, ['equipo'], how='left').show(100)

+--------------------+-----------------+
|              equipo|   Promedio Total|
+--------------------+-----------------+
|Servicios a domic...|4.424074074074074|
|Lo tengo y te lo ...| 4.47699530516432|
|Gestión de Reside...|4.470531400966183|
|       Te lo cambio.|4.285446009389672|
|Administración de...|4.187619047619048|
|Deserción de estu...|4.766666666666667|
|Gestión de Menús ...|4.355251141552512|
|Gestión de progra...|              4.4|
|     Mi Sanandresito|4.202739726027397|
+--------------------+-----------------+

+--------------------+--------------------+-----------------+
|              equipo|               email|   Promedio Total|
+--------------------+--------------------+-----------------+
|Administración de...|lumarinb@unal.edu.co|4.187619047619048|
|Administración de...|framirezf@unal.ed...|4.187619047619048|
|Administración de...|samolinap@unal.ed...|4.187619047619048|
|Administración de...|sednarvaezna@unal...|4.187619047619048|
|Administración de...|nsalinas@u